In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
DEBUG=False

In [ ]:
%run ./nb_core/root_dirs.ipynb
setup_syspath_disentangle(DEBUG)
%run ./nb_core/disentangle_imports.ipynb

In [ ]:
nmodel_dir = '/home/ashesh.ashesh/training/noise_model/2402/139'
# nmodel_dir = '/home/ashesh.ashesh/training/noise_model/2402/61'

histnoisemodel_fpath = None
gmmnoisemodel_fpath = None
for fname in os.listdir(nmodel_dir):
    if fname.startswith('HistNoiseModel'):
        assert histnoisemodel_fpath is None
        histnoisemodel_fpath = os.path.join(nmodel_dir, fname)
    elif fname.startswith('GMMNoiseModel'):
        assert gmmnoisemodel_fpath is None
        gmmnoisemodel_fpath = os.path.join(nmodel_dir, fname)
print(gmmnoisemodel_fpath)
print(histnoisemodel_fpath)

In [ ]:
from disentangle.nets.gmm_noise_model import GaussianMixtureNoiseModel
from disentangle.nets.hist_noise_model import HistNoiseModel

# gmmnoisemodel_fpath = '/home/ashesh.ashesh/training/noise_model/2402/62/GMMNoiseModel_CCPs-GT_all.mrc__6_4_Clip0.0-1.0_Sig0.125_UpNone_Norm0_bootstrap.npz'
# histnoisemodel_fpath = os.path.join(os.path.dirname(gmmnoisemodel_fpath), 'HistNoiseModel_CCPs-GT_all.mrc__Norm0_Bins128_bootstrap.npy')
# datadir = '/group/jug/ashesh/data/ventura_gigascience/actin-60x-noise2-highsnr.tif' if 'actin' in os.path.basename(gmmnoisemodel_fpath) else '/group/jug/ashesh/data/ventura_gigascience/mito-60x-noise2-highsnr.tif'


In [ ]:
nmodel_params = np.load(gmmnoisemodel_fpath)
gmm_model = GaussianMixtureNoiseModel(params=nmodel_params)
histdata = np.load(histnoisemodel_fpath)
hist_model = HistNoiseModel(histdata)
bins = histdata.shape[-1]

In [ ]:
from disentangle.utils import plotProbabilityDistribution
data_dict = plotProbabilityDistribution(signalBinIndex=80, 
                            histogramNoiseModel=hist_model,
                            gaussianMixtureNoiseModel=gmm_model,
                            device='cpu')

In [ ]:
for i in range(histdata.shape[1]):
    assert np.std(histdata[1][i]) < 1e-7
    assert np.std(histdata[2][i]) < 1e-7

In [ ]:
bin_val = (histdata[1] + histdata[2])/2
bin_val = bin_val[:,0]
binsize = np.mean(histdata[2] - histdata[1])
bin_pdf = histdata[0]/binsize

In [ ]:
from scipy.optimize import curve_fit
import math
import numpy as np

def gaus(x, mu,sigma):
    out = np.exp(-(x-mu)**2/(2*sigma**2)) * 1/(sigma*np.sqrt(2*math.pi))
    # print(out.shape, out.min(), out.max())
    return out

In [ ]:
index = 64
x = bin_val
y = bin_pdf[index]

mean =bin_val[index]
sigma = sum(y*(x-mean)**2)/len(y)

popt,pcov = curve_fit(gaus,
                      np.array(bin_val),
                      np.array(bin_pdf[index]),
                      p0=[bin_val[index],100])


In [ ]:
popt

In [ ]:
plt.plot(bin_val,bin_pdf[index],'b+:',label='data')
plt.plot(bin_val,gaus(bin_val,*popt),'ro:',label='fit')
plt.legend()
plt.title('Fig. 3 - Fit for Time Constant')
plt.xlabel('Time (s)')
plt.ylabel('Voltage (V)')
plt.show()

In [ ]:
# import pylab as plb
# import matplotlib.pyplot as plt
# from scipy.optimize import curve_fit
# from scipy import asarray as ar,exp

# x = ar(range(10))
# y = ar([0,1,2,3,4,5,4,3,2,1])

# n = len(x)                          #the number of data
# mean = sum(x*y)/n                   #note this correction
# sigma = sum(y*(x-mean)**2)/n        #note this correction

# def gaus(x,a,x0,sigma):
#     return a*exp(-(x-x0)**2/(2*sigma**2))

# popt,pcov = curve_fit(gaus,x,y,p0=[1,mean,sigma])

# plt.plot(x,y,'b+:',label='data')
# plt.plot(x,gaus(x,*popt),'ro:',label='fit')
# plt.legend()
# plt.title('Fig. 3 - Fit for Time Constant')
# plt.xlabel('Time (s)')
# plt.ylabel('Voltage (V)')
# plt.show()

In [ ]:
y.shape

In [ ]:
gmm_model.getGaussianParameters(torch.Tensor([5371]))

In [ ]:
for k in range(0, histdata.shape[1], 5):
    print(f'{histdata[1,k,0]:.1f} {histdata[2,k,0]:.1f}')

In [ ]:
34643.6 -33710.7

In [ ]:
81972.1 - 82905.1
